<a href="https://colab.research.google.com/github/AryanKohli88/AryanKohli88/blob/main/Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle

In [ ]:
# upload kaggle.json

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# importing dataset

In [ ]:
!kaggle datasets download -d kazanova/sentiment140

 82% 66.0M/80.9M [00:00<00:00, 129MB/s]
100% 80.9M/80.9M [00:00<00:00, 136MB/s]


In [ ]:
from zipfile import ZipFile
dataset = '/content/sentiment140.zip'

with ZipFile(dataset, 'r') as zip:
  zip.extractall()
  print('extraction done')

extraction done


In [ ]:
# importing dependencies

In [ ]:
import numpy as np
import pandas as pd
import re

In [ ]:
from nltk.corpus import stopwords

In [ ]:
from nltk.stem.porter import PorterStemmer

In [ ]:
from  sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
#cleaning text

import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv', encoding= "ISO-8859-1")

In [ ]:
twitter_data.shape

(1599999, 6)

In [ ]:
twitter_data.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [ ]:
# naming the columns
col_names = ['target', 'id', 'date', 'flag', 'user', 'text']

In [ ]:
twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv', names = col_names, encoding= "ISO-8859-1")

In [ ]:
twitter_data.shape

(1600000, 6)

In [ ]:
twitter_data.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [ ]:
twitter_data.isnull().sum()

target    0
id        0
date      0
flag      0
user      0
text      0
dtype: int64

In [ ]:
# checking the distribution of target column
twitter_data['target'].value_counts()

0    800000
4    800000
Name: target, dtype: int64

In [ ]:
twitter_data.replace({'target':{4:1}}, inplace=True)
twitter_data['target'].value_counts()

0    800000
1    800000
Name: target, dtype: int64

In [ ]:
# 0 -> negative emotion
# 1 -> positive emotion

In [ ]:
potstem = PorterStemmer()
def stemming(content):
  stemmed_content = re.sub('[^a-zA-Z]',' ', content)
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  stemmed_content = [potstem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content = ' '.join(stemmed_content)

  return stemmed_content

In [ ]:
twitter_data['stemmed_content'] = twitter_data['text'].apply(stemming)

In [ ]:
twitter_data.head()

,target,id,date,flag,user,text,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot http twitpic com zl awww bummer sho...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad see


In [45]:
X = twitter_data['stemmed_content'].values
Y = twitter_data['target'].values

X_train, X_test, Y_train, Y_test =  train_test_split(X, Y, test_size = 0.2, stratify=Y, random_state=2)

vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(X_train) # fit only training data
X_test = vectorizer.transform(X_test)



Training the model

In [ ]:
model = LogisticRegression(max_iter=1000)
model.fit(X_train, Y_train)

LogisticRegression(max_iter=1000)

Model Evaluation

In [ ]:
X_train_predic = model.predict(X_train)
training_data_acc = accuracy_score(Y_train, X_train_predic)
print('training accuracy is :: ', training_data_acc)

X_test_predic = model.predict(X_test)
testing_data_acc = accuracy_score(Y_test, X_test_predic)
print('testing accuracy is :: ', testing_data_acc)


training accuracy is ::  0.81018984375
testing accuracy is ::  0.7780375


In [44]:
import pickle
filename = 'trained_sentiments.pkl'
pickle.dump(model, open(filename, 'wb'))

In [46]:
loadedmodel = pickle.load(open('/content/trained_sentiments.pkl', 'rb'))

In [54]:
Xnew = X_test[3]
print(twitter_data['text'][3])

prediction = loadedmodel.predict(Xnew)
print(prediction)
if(prediction[0] == 0):
  print("Negative")
else:
  print("Positive")

my whole body feels itchy and like its on fire 
[0]
Negative


In [55]:
Xnew = X_test[200]
print(twitter_data['text'][200])

prediction = loadedmodel.predict(Xnew)
print(prediction)
if(prediction[0] == 0):
  print("Negative")
else:
  print("Positive")

Job Interview in Cardiff today, wish me luck! Got about 3 hours sleep 
[1]
Positive
